In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [4]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_seasonal.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_seasonal.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_seasonal.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_prcptot_seasonal.nc']

In [5]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_prcptot_seasonal.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds45 = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds45.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 152)
Coordinates:
  * time         (time) datetime64[ns] 1949-12-01 1950-12-01 ... 2100-12-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 3, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

In [13]:
i = 0
df45list = []
for ds in listds45:
    print ("Dataset number: ", i)
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    dfp452 =  df453.reset_index()
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    Region1i45["season"] = season
    df45list.append(Region1i45)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list


In [23]:
df45list[1]

,index,lat,lon,time,prcptot_p10,prcptot_p50,prcptot_p90,season
0,0,66.623306,-89.045212,1950-03-01,26.24,34.04,48.76,spring
1,1,66.623306,-89.045212,1951-03-01,19.19,28.68,50.52,spring
2,2,66.623306,-89.045212,1952-03-01,22.87,32.68,43.02,spring
3,3,66.623306,-89.045212,1953-03-01,20.97,29.12,44.52,spring
4,4,66.623306,-89.045212,1954-03-01,32.82,35.76,47.81,spring
...,...,...,...,...,...,...,...,...
11507101,220720470,40.041039,-74.045807,2096-03-01,252.86,355.24,379.51,spring
11507102,220720471,40.041039,-74.045807,2097-03-01,281.21,335.83,441.66,spring
11507103,220720472,40.041039,-74.045807,2098-03-01,303.70,363.45,390.07,spring
11507104,220720473,40.041039,-74.045807,2099-03-01,235.84,336.19,389.11,spring


In [25]:
df45 = pd.concat(df45list)

# FOR RCP85 

In [9]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 152)
Coordinates:
  * time         (time) datetime64[ns] 1949-12-01 1950-12-01 ... 2100-12-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 3, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

In [15]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df853 = df852.loc[df852["realization"] == 0].dropna()
    print ("Converted to DF")
    dfp852 =  df853.reset_index()
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"],2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"],2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    Region1i85["season"] = season
    df85list.append(Region1i85)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list


In [16]:
df85list[0]

,index,lat,lon,time,prcptot_p10,prcptot_p50,prcptot_p90,season
0,1,66.623306,-89.045212,1950-12-01,24.52,34.35,47.57,winter
1,2,66.623306,-89.045212,1951-12-01,22.42,30.76,46.76,winter
2,3,66.623306,-89.045212,1952-12-01,16.10,29.69,50.23,winter
3,4,66.623306,-89.045212,1953-12-01,18.61,25.36,34.17,winter
4,5,66.623306,-89.045212,1954-12-01,24.09,33.69,44.41,winter
...,...,...,...,...,...,...,...,...
11430895,222182194,40.041039,-74.045807,2095-12-01,275.28,418.35,452.01,winter
11430896,222182195,40.041039,-74.045807,2096-12-01,191.37,335.91,407.47,winter
11430897,222182196,40.041039,-74.045807,2097-12-01,244.92,332.17,465.98,winter
11430898,222182197,40.041039,-74.045807,2098-12-01,228.98,350.62,390.72,winter


In [24]:
df85 = pd.concat(df85list)

# OBSERVED DATA

In [17]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [18]:
dsOBs

<xarray.Dataset>
Dimensions:  (lat: 510, lon: 1068, time: 257)
Coordinates:
  * time     (time) datetime64[ns] 1949-12-01 1950-03-01 ... 2013-12-01
  * lat      (lat) float32 83.45833 83.375 83.291664 ... 41.125 41.041668
  * lon      (lon) float32 -140.95833 -140.875 ... -52.125004 -52.04167
Data variables:
    prcptot  (time, lat, lon) float32 dask.array<chunksize=(257, 510, 1068), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           NRCAN 10km Gridded Climate Dataset
    history:         2012-10-22T13:14:41: Convert from original format to Net...
    institution:     NRCAN
    source:          ANUSPLIN
    redistribution:  Redistribution policy unknown. For internal use only.

## Convert in Dataframe

In [19]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [20]:
dfObs = dfmeanObs.rename(columns={var: var+'_Obs'})

In [21]:
dfObs

,lat,lon,time,prcptot_Obs
0,83.125000,-74.541672,1950-03-01,16.820000
1,83.125000,-74.541672,1950-06-01,20.840002
2,83.125000,-74.541672,1950-09-01,50.200008
3,83.125000,-74.541672,1950-12-01,20.729998
4,83.125000,-74.541672,1951-03-01,16.969997
...,...,...,...,...
65602060,41.708332,-82.208336,2012-09-01,166.379974
65602061,41.708332,-82.208336,2012-12-01,208.669983
65602062,41.708332,-82.208336,2013-03-01,190.270004
65602063,41.708332,-82.208336,2013-06-01,350.260071


In [ ]:
dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [ ]:
dfObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [26]:
df2m =  pd.merge(df45, df85, on=["lat","lon", "time"], suffixes=('_45', '_85'))

In [27]:
df2m

,index_45,lat,lon,time,prcptot_p10_45,prcptot_p50_45,prcptot_p90_45,season_45,index_85,prcptot_p10_85,prcptot_p50_85,prcptot_p90_85,season_85
0,1,66.623306,-89.045212,1950-12-01,25.03,34.60,47.51,winter,1,24.52,34.35,47.57,winter
1,2,66.623306,-89.045212,1951-12-01,22.80,30.75,46.68,winter,2,22.42,30.76,46.76,winter
2,3,66.623306,-89.045212,1952-12-01,16.67,29.58,50.60,winter,3,16.10,29.69,50.23,winter
3,4,66.623306,-89.045212,1953-12-01,19.05,25.30,34.58,winter,4,18.61,25.36,34.17,winter
4,5,66.623306,-89.045212,1954-12-01,24.16,33.77,44.65,winter,5,24.09,33.69,44.41,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45952213,220720470,40.041039,-74.045807,2096-09-01,200.24,284.72,424.18,fall,220720470,206.37,312.75,359.91,fall
45952214,220720471,40.041039,-74.045807,2097-09-01,214.80,299.52,394.04,fall,220720471,179.89,259.75,451.88,fall
45952215,220720472,40.041039,-74.045807,2098-09-01,185.18,218.00,376.83,fall,220720472,138.02,273.69,396.32,fall
45952216,220720473,40.041039,-74.045807,2099-09-01,221.48,326.66,357.05,fall,220720473,211.11,303.41,379.98,fall


# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [28]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [29]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [30]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [31]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [32]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [33]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [34]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [35]:
dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 


NameError: name 'dfdataregObs' is not defined

In [ ]:
dfmeanObs

In [38]:
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/8... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/8... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"])
    print ("3/8... Get mean by year for every sub-region")
    dfmeanUA45 = dfdatareg45
    dfmean45UA = dfmeanUA45.groupby(["time", name]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 
    dfmeanObs2 = dfmeanObs.reset_index()
    print("Merging 3 cases")
    df3m = pd.merge(dfmean45UA2, dfmeanObs2, how='left', on= ["time", name] )
    print ("4/8... Split by subregion")
    df3ma = df3m[['time', name, var+'_p10_45', var+'_p50_45', var+'_p90_45',
                               var+'_p10_85', var+'_p50_85', var+'_p90_85', var+'_Obs']]
    if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15
    if (var == "solidprcptot"):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]/10
    df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)
    for name2,dfsr in df3ma.groupby(name):
        print ("                   "+name2, name)
        dfsrc = dfsr.copy()
        print ("6/8... 4th Polynomial - rcp45,85")
        years = dfsrc.time.dt.year
        dfsrc["time"] = years#.values
        for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
            if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4) - 273.15
            if (var == "solidprcptot"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)/10
            dfsrc[var+p] = round(dfsrc[var+p], 2) 
        print(" -----------------------8/8 Creating CSV file for sub-region " +name2)
        dfsrc.to_csv(replace_all(name2,d)+"_"+var+"_annual.csv")
    
    
                            
       

------------------------------------------------------------------
1/8... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/8... Merging region with data
3/8... Get mean by year for every sub-region


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)



Merging 3 cases
4/8... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ABITIBI-TEMISCAMINGUE
                   BAS-SAINT-LAURENT NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region BAS-SAINT-LAURENT
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
                   COTE-NORD NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region COTE-NORD
                   ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
                   GASPESIE-ILES-DE-LA-MADELEINE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85

                   11161 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 11161
                   11262 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 11262
                   11263 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 11263
------------------------------------------------------------------
1/8... Cutting by region
DOM_BIO NOM
2/8... Merging region with data
3/8... Get mean by year for every sub-region
Merging 3 cases
4/8... Split by subregion
                   Golfe du Saint-Laurent NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Golfe du Saint-Laurent
                   Pessière à lichens NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Pessière à lichens
                   Pessière à mousses NOM
6/8... 4th P

                   Îles-de-la-Madeleine NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Îles-de-la-Madeleine
------------------------------------------------------------------
1/8... Cutting by region
SDOM_BIO NOM
2/8... Merging region with data
3/8... Get mean by year for every sub-region
Merging 3 cases
4/8... Split by subregion
                   Golfe du Saint-Laurent NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Golfe du Saint-Laurent
                   Pessière à mousses de l'Est NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Pessière à mousses de l'Est
                   Pessière à mousses de l'Ouest NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Pessière à mousses de l'Ouest
                   Sapinière à bouleau blanc de l'Est NOM
6/8... 4th Polynomial - rcp45,85
 ------------

 -----------------------8/8 Creating CSV file for sub-region Golfe du Saint-Laurent
                   Hautes collines de Baie-Comeau-Sept-Îles NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Hautes collines de Baie-Comeau-Sept-Îles
                   Hautes collines de Saint-Tite-des-Caps NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Hautes collines de Saint-Tite-des-Caps
                   Hautes collines de Val-David-Lac-Mékinac NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Hautes collines de Val-David-Lac-Mékinac
                   Hautes collines des lacs Nipissis et Magpie NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Hautes collines des lacs Nipissis et Magpie
                   Hautes collines des lacs Walker et Beetz NOM
6/8... 4th Polynomial - rcp45,85
 --------------------